In [1]:
import pandas as pd
import numpy as np
import psycopg2
import query_maker
np.random.seed(42)
# exams = pd.read_csv('exams.csv')
tb_city = pd.read_csv('db/City.csv')
tb_country = pd.read_csv('db/Country.csv')

In [2]:
# print(exams.head(1))
# print(exams.columns)
print(tb_city.columns)
print(tb_country.columns)

Index(['id', 'country_code', 'population'], dtype='object')
Index(['code', 'surface_area', 'indep_year', 'population', 'life_expectancy',
       'gnp', 'gnp_old', 'capital'],
      dtype='object')


In [3]:
# exams.drop(['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course'], axis=1, inplace=True)

In [4]:
# Use an external config.py file instead to store the credentials
import config
print(config.hostname)

# Copy this (or something similar) into config.py:
# hostname = 'localhost'
# database = 'postgres'
# username = 'postgres'
# pwd = None # need to add this
# port_id = 5433

localhost


In [5]:
tables = [tb_city, tb_country]
table_names = ['city', 'country']
cols_names = ['id', 'population', 'surface_area', 'indep_year', 'population', 
       'life_expectancy','gnp', 'gnp_old', 'capital']

In [6]:
sqlQuery, vectorQuery = query_maker.generate_queries(tables, 
                                                     table_names,n=20000, 
                                                     max_nr_preds = 2, 
                                                     cols_names=cols_names)

In [7]:
plan_results = []
actual_results = []

In [8]:
try:
  # need to download postgres and supply your own info 
  # with password and stuff
  connection = psycopg2.connect(host = config.hostname,
                                dbname = config.database,
                                user = config.username,
                                password = config.pwd,
                                port = config.port_id)

  cur = connection.cursor()

  for query in sqlQuery:
    cur.execute('explain (format json, analyze) ' + query)
    result = cur.fetchone()[0][0]
    plan_results.append(result['Plan']['Plan Rows'])
    actual_results.append(result['Plan']['Actual Rows'])

except Exception as error:
  print(error)
finally:
  if cur:
    cur.close()
  if connection:
    connection.close()


In [9]:
plan_results

[1,
 1,
 1,
 1229,
 693,
 6,
 32,
 4078,
 13,
 2889,
 111,
 101,
 230,
 3942,
 819,
 444,
 3942,
 4068,
 174,
 3021,
 167,
 1,
 2,
 55,
 4078,
 17,
 4078,
 4078,
 3874,
 2,
 3721,
 1060,
 1,
 1399,
 102,
 34,
 1,
 4054,
 33,
 17,
 85,
 160,
 1,
 1626,
 1,
 205,
 177,
 160,
 17,
 1,
 3980,
 20,
 1,
 67,
 1,
 1,
 17,
 2634,
 17,
 2194,
 3874,
 4060,
 17,
 1,
 4078,
 3942,
 798,
 17,
 37,
 3021,
 1,
 1,
 177,
 89,
 17,
 17,
 222,
 4051,
 2,
 3942,
 3843,
 4078,
 95,
 2543,
 3021,
 234,
 30,
 39,
 17,
 2999,
 90,
 4055,
 1119,
 3448,
 4042,
 2400,
 177,
 4078,
 1,
 4076,
 2987,
 495,
 17,
 17,
 1,
 2887,
 4075,
 2082,
 4043,
 4008,
 4058,
 45,
 3021,
 178,
 25,
 3021,
 23,
 1297,
 178,
 1,
 3016,
 4079,
 74,
 495,
 4072,
 4059,
 90,
 222,
 256,
 3603,
 754,
 1724,
 2120,
 4078,
 1,
 38,
 2827,
 567,
 2529,
 4078,
 1,
 3942,
 2987,
 3567,
 85,
 3021,
 1571,
 3021,
 3884,
 3021,
 137,
 1,
 4078,
 2152,
 4078,
 4078,
 713,
 3150,
 176,
 17,
 461,
 17,
 3021,
 1,
 991,
 17,
 1,
 3943,
 99,
 30

In [10]:
sqlQuery

['SELECT * FROM country WHERE country.gnp_old = 1692900',
 'SELECT * FROM city WHERE city.population = 9958656',
 'SELECT * FROM city WHERE city.id = 3381 AND city.population != 4981547',
 'SELECT * FROM city, country WHERE country_code = code AND country.capital < 1216',
 'SELECT * FROM city, country WHERE country_code = code AND city.id >= 3386',
 'SELECT * FROM city, country WHERE country_code = code AND city.population > 9305690',
 'SELECT * FROM city WHERE city.population >= 4721381',
 'SELECT * FROM city, country WHERE country_code = code AND city.population != 84696',
 'SELECT * FROM city, country WHERE country_code = code AND city.population >= 8242722',
 'SELECT * FROM city WHERE city.id <= 2889',
 'SELECT * FROM country WHERE country.gnp_old != 2795670 AND country.capital < 2825',
 'SELECT * FROM country WHERE country.capital >= 2557 OR country.gnp_old >= 4998052',
 'SELECT * FROM country WHERE country.capital < 3944',
 'SELECT * FROM city, country WHERE country_code = code A

In [13]:
vectorQuery_df = pd.DataFrame(vectorQuery)
vectorQuery_df['actual_cardinality'] = actual_results
vectorQuery_df['postgres_cardinality'] = plan_results
vectorQuery_df['sql_query'] = sqlQuery
vectorQuery_df

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,actual_cardinality,postgres_cardinality,sql_query
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,SELECT * FROM country WHERE country.gnp_old = ...
1,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,SELECT * FROM city WHERE city.population = 995...
2,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,4981547,1,1,SELECT * FROM city WHERE city.id = 3381 AND ci...
3,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1364,1229,"SELECT * FROM city, country WHERE country_code..."
4,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,694,693,"SELECT * FROM city, country WHERE country_code..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,232,231,SELECT * FROM country WHERE country.capital !=...
19996,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3209,2970,"SELECT * FROM city, country WHERE country_code..."
19997,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,3053,48,47,SELECT * FROM country WHERE country.capital < ...
19998,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,37,38,SELECT * FROM city WHERE city.population >= 36...


In [14]:
vectorQuery_df.to_csv('vectorizedQueries.csv')